In [1519]:
import pandas as pd
import numpy as np

In [1520]:
#defining bias and input values
BIT_WIDTH = int(8)
num_inputs = 4           
is_signed = True          
is_sigmoid_signed = False 
weights_on_memory = True                #1: on memory || 0: weights = inputs
n_integer = 1                           # 'm' binary digits are the integer part
n_mantissa = BIT_WIDTH - n_integer      # 'x' binary digits are the fractional part of `x`;
# n_mantissa_sigmoid = n_mantissa-1       # casas decimais
n_mantissa_sigmoid = n_mantissa         # casas decimais
memory_address_type = True              # 0= int(n) ; 1= float(address)|| indice da memoria int ou float

integer_mult_factor = 1
sigmoid_range = 2  #tamanho eixo x, sigmoide antes tratamento
range_tb_df = (2**n_integer)            #tamanho do dataframe do tb
# mult_factor = 10                         # expande sigmoide
integer_mult_factor = 2** (BIT_WIDTH-1) # conversão para inteiros
# integer_mult_factor = 1 # conversão para inteiros

DOWNLOAD = True                         #1= para baixar || 0= não baixar
path = "./"


In [1521]:
n_mantissa

7

In [1522]:
n_integer

1

In [1523]:
range_tb_df = (2**n_integer)              #tamanho do dataframe do tb
#DX SIGMOID
is_signed = True           #1= signed || 0= unsigned
if is_signed:
  unsigned_offset = 0
  offset_df_memory = 0
  dx_sigmoid_final = 1/(2**(n_mantissa_sigmoid))     #dx da sigmoid na memória -> depende das casas .fixo com adicional para resolução de dx_input
  
else:
  unsigned_offset = int((2**n_integer)/2)  # 128
  offset_df_memory = unsigned_offset - ((sigmoid_range/2)*integer_mult_factor)
  dx_sigmoid_final = 1/(2**(n_mantissa_sigmoid))     #dx da sigmoid na memória -> depende das casas .fixo com adicional para resolução de dx_input

if is_sigmoid_signed: # normalizando para utilizar todos os bits
  norm_y = 1
else:
  norm_y = (2**(n_integer))                   # normalização em y: 256(8bits)
# dx_sigmoid = dx_sigmoid_final/mult_factor   #dx necessário antes da expansão
# dx_sigmoid = dx_sigmoid_final/integer_mult_factor   #dx necessário antes da expansão
dx_sigmoid = dx_sigmoid_final   #dx necessário antes da expansão

if (dx_sigmoid == 1):
  total_dx = range_tb_df+1
else:
  
  total_dx = (1/dx_sigmoid)*sigmoid_range +1

In [1524]:
sigmoid_dx_list = np.linspace(-(sigmoid_range/2),(sigmoid_range/2),int(total_dx)) #vai de -128 a 127, dividido em 256 pontos (8 bits)


In [1525]:
df = pd.DataFrame(np.linspace(-(sigmoid_range/2),(sigmoid_range/2),int(total_dx))) #start, end, step
df.columns = ['x'] # INPUTS_df
df['x_resc'] = df*integer_mult_factor
# df.head()
df

,x,x_resc
0,-1.000000,-128.0
1,-0.992188,-127.0
2,-0.984375,-126.0
3,-0.976562,-125.0
4,-0.968750,-124.0
...,...,...
252,0.968750,124.0
253,0.976562,125.0
254,0.984375,126.0
255,0.992188,127.0


In [1526]:
import math
exp = math.exp

In [1793]:
# sigmoid_width_rescale = 0.8
sigmoid_width_rescale = 0.7
# sigmoid_width_rescale = 0.6
# sigmoid_width_rescale = 0.5
# sigmoid_width_rescale = 0.4
sigmoide_rescale = (8/(sigmoid_range/2))* sigmoid_width_rescale

# sigmoide_rescale = 1
sigmoide_rescale

5.6

In [1794]:
# (1/(1+(e^(-1*(x*sigmoide_rescale)))))
multiplied_list = [(1/(1+(exp(-1*(element*sigmoide_rescale))))) for element in sigmoid_dx_list]
# multiplied_list = [(1/(1+(exp(-1*(element*sigmoide_rescale)))) *(1.1)) for element in sigmoid_dx_list]
df['sigmoid']= multiplied_list
df['sigmoid_resc']= df['sigmoid']*integer_mult_factor
df

,x,x_resc,sigmoid,sigmoid_resc,sig_norm
0,-1.000000,-128.0,0.003684,0.471583,0.424653
1,-0.992188,-127.0,0.003848,0.492591,0.446388
2,-0.984375,-126.0,0.004020,0.514532,0.469232
3,-0.976562,-125.0,0.004199,0.537446,0.493244
4,-0.968750,-124.0,0.004386,0.561376,0.518482
...,...,...,...,...,...
252,0.968750,124.0,0.995614,127.438624,255.481518
253,0.976562,125.0,0.995801,127.462554,255.506756
254,0.984375,126.0,0.995980,127.485468,255.530768
255,0.992188,127.0,0.996152,127.507409,255.553612


## Sigmoid expansion (change 'x' rescale)

In [1795]:
# df['x'] = n_re_scale
# df['n_zeros'] = (df['x'] + dx_sigmoid) #zeros para serem salvos na memória
df['sig_norm']= df['sigmoid_resc']*norm_y
df

,x,x_resc,sigmoid,sigmoid_resc,sig_norm
0,-1.000000,-128.0,0.003684,0.471583,0.943165
1,-0.992188,-127.0,0.003848,0.492591,0.985183
2,-0.984375,-126.0,0.004020,0.514532,1.029064
3,-0.976562,-125.0,0.004199,0.537446,1.074892
4,-0.968750,-124.0,0.004386,0.561376,1.122751
...,...,...,...,...,...
252,0.968750,124.0,0.995614,127.438624,254.877249
253,0.976562,125.0,0.995801,127.462554,254.925108
254,0.984375,126.0,0.995980,127.485468,254.970936
255,0.992188,127.0,0.996152,127.507409,255.014817


## df_bin

In [1796]:
from fxpmath import Fxp
#SIGMOID MEMORY
df_bin = pd.DataFrame()

# X
df_bin['address'] = (df['x_resc'])
df_bin['address_bin'] = (df_bin['address'].apply(lambda x:  Fxp(x, signed=is_signed, n_word= BIT_WIDTH, n_frac= 0).bin() ) )
address_bin_converted = df_bin['address_bin'].apply(lambda x:  Fxp(f"0b{x}", signed=is_signed, n_word= BIT_WIDTH, n_frac= 0).astype(float) ) 
df_bin['address_bin_conv'] = address_bin_converted
df_bin

,address,address_bin,address_bin_conv
0,-128.0,10000000,-128.0
1,-127.0,10000001,-127.0
2,-126.0,10000010,-126.0
3,-125.0,10000011,-125.0
4,-124.0,10000100,-124.0
...,...,...,...
252,124.0,01111100,124.0
253,125.0,01111101,125.0
254,126.0,01111110,126.0
255,127.0,01111111,127.0


In [1797]:
# Y (sigmoid)
df_bin['sigmoid'] = (df['sig_norm'])
df_bin['sigmoid_bin'] = (df_bin['sigmoid'].apply(lambda x:  Fxp(x, signed=is_sigmoid_signed, n_word= BIT_WIDTH, n_frac= 0).bin() ) )
df_bin['sigmoid_bin_conv'] = df_bin['sigmoid_bin'].apply(lambda x:  Fxp(f"0b{x}", signed=is_sigmoid_signed, n_word= BIT_WIDTH, n_frac= 0).astype(float) ) 
df_bin

,address,address_bin,address_bin_conv,sigmoid,sigmoid_bin,sigmoid_bin_conv
0,-128.0,10000000,-128.0,0.943165,00000000,0.0
1,-127.0,10000001,-127.0,0.985183,00000000,0.0
2,-126.0,10000010,-126.0,1.029064,00000001,1.0
3,-125.0,10000011,-125.0,1.074892,00000001,1.0
4,-124.0,10000100,-124.0,1.122751,00000001,1.0
...,...,...,...,...,...,...
252,124.0,01111100,124.0,254.877249,11111110,254.0
253,125.0,01111101,125.0,254.925108,11111110,254.0
254,126.0,01111110,126.0,254.970936,11111110,254.0
255,127.0,01111111,127.0,255.014817,11111111,255.0


In [1798]:
# # AJUSTANTO DATAFRAME CONFORME LIMITES DE REPRESENTAÇÃO
# upper_limit = (sigmoid_range/2 - (1/2**(BIT_WIDTH-1))) * (2**(BIT_WIDTH-1))
# low_limit = -(sigmoid_range/2) * (2**(BIT_WIDTH-1))
# print(upper_limit,low_limit)

# df_bin['sigmoid'].loc[df_bin['sigmoid'] > upper_limit] = upper_limit # substituindo valores maiores que o limite superior, pelo próprio limite superior
# df_bin['address'].loc[df_bin['address'] > upper_limit] = upper_limit # substituindo valores maiores que o limite superior, pelo próprio limite superior


df_bin.drop(df_bin.tail(1).index,inplace=True) # drop last n rows
df_bin

,address,address_bin,address_bin_conv,sigmoid,sigmoid_bin,sigmoid_bin_conv
0,-128.0,10000000,-128.0,0.943165,00000000,0.0
1,-127.0,10000001,-127.0,0.985183,00000000,0.0
2,-126.0,10000010,-126.0,1.029064,00000001,1.0
3,-125.0,10000011,-125.0,1.074892,00000001,1.0
4,-124.0,10000100,-124.0,1.122751,00000001,1.0
...,...,...,...,...,...,...
251,123.0,01111011,123.0,254.827268,11111110,254.0
252,124.0,01111100,124.0,254.877249,11111110,254.0
253,125.0,01111101,125.0,254.925108,11111110,254.0
254,126.0,01111110,126.0,254.970936,11111110,254.0


In [1799]:
#conferir quantas linhas de endereço duplicadas tem
len(df_bin['address_bin'])-len(df_bin['address_bin'].drop_duplicates())

0

# Graphs

In [1800]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import plotly.offline as py

In [1801]:
x_axis = df_bin['address']
y_axis = df_bin['sigmoid_bin_conv']
pacitydef = 0.8

# sigmoid = go.Scatter(
#     x= x_axis,
#     y=df_bin['sigmoid'],
#     name="sigmoid",
#     # line= dict(color='#FF8000'),
#     opacity = 0.8,
#     line_shape='spline')

sigmoid_bin = go.Scatter(
    x= x_axis,
    y= y_axis,
    name="sigmoid_bin",
    # line= dict(color='#FF8000'),
    opacity = 0.8,
    line_shape='spline')

# dadospred = ([sigmoid,sigmoid_bin])
dadospred = ([sigmoid_bin])
figpred = go.Figure(data=dadospred)#, layout=layout)
py.iplot(figpred)

In [1802]:
df_bin

,address,address_bin,address_bin_conv,sigmoid,sigmoid_bin,sigmoid_bin_conv
0,-128.0,10000000,-128.0,0.943165,00000000,0.0
1,-127.0,10000001,-127.0,0.985183,00000000,0.0
2,-126.0,10000010,-126.0,1.029064,00000001,1.0
3,-125.0,10000011,-125.0,1.074892,00000001,1.0
4,-124.0,10000100,-124.0,1.122751,00000001,1.0
...,...,...,...,...,...,...
251,123.0,01111011,123.0,254.827268,11111110,254.0
252,124.0,01111100,124.0,254.877249,11111110,254.0
253,125.0,01111101,125.0,254.925108,11111110,254.0
254,126.0,01111110,126.0,254.970936,11111110,254.0


In [1869]:
#colocando na memória com o Index
df_sigmoid_first= float(df['x'].iloc[:1])
i= 0
text = str('')
text_zeros = str('')
#texto para fazer lógica 'signed' funcionar no std_logic_vector
text_rom_to_asic = str('--"int_f_x",--(address)  =integer_MAC|| f(x)               = int_f_x \n')# 0 a 1 e -1 a 0

#------------------------------- ROM ASIC -------------------------------#
#texto rom -> vhd (asic) || índice por 'x'
for i in range(int(len(df_bin)/2), len(df_bin)): 
    text_rom_to_asic = (
        text_rom_to_asic+'"' 
             +str(df_bin['sigmoid_bin'][i])+  '",-- '
             +f"({(df_bin['address_bin'][i])})"
             +f" = {str(df_bin['address_bin_conv'][i]).ljust(9, ' ')}"
             +f" || {str(df['sig_norm'][i]).ljust(18, ' ')} = {str(df_bin['sigmoid_bin_conv'][i])}\n")

for i in range(0, int(len(df_bin)/2) ): 
    text_rom_to_asic = (
        text_rom_to_asic+'"' #pega o q já tem de texto
             +str(df_bin['sigmoid_bin'][i])+  '",-- ' #pega elemento i da sigmoide
             +f"({df_bin['address_bin'][i]})"
             +f" = {str(df_bin['address_bin_conv'][i]).ljust(9, ' ')}"
             +f" || {str(df['sig_norm'][i]).ljust(18, ' ')} = {str(df_bin['sigmoid_bin_conv'][i])}\n")
    

#------------------------------- ROM .mif -------------------------------#
#define se o endereço será com 'x' ou 'address'
if (memory_address_type == False): # 0= int(n)
  for i in range(0, len(df_bin)): #caso seja por 'x'
    text = (text+''+ str(df_bin.index[i]) + ':' + str(df_bin['sigmoid_bin'][i])+  ';-- '+str(offset_df_memory +df_sigmoid_first+(i*dx_sigmoid*integer_mult_factor))+' || '+str(df['sig_norm'][i]) +'\n')
    # )
else: #1= float(address)
  for i in range(0, len(df_bin)):
    text = (text+''+ str(df_bin['address_bin'][i]) + ':' +str(df_bin['sigmoid_bin'][i])+  ';-- '+str(offset_df_memory +df_sigmoid_first+(i*dx_sigmoid*integer_mult_factor))+' || '+str(df['sig_norm'][i]) +'\n')
  

## ROM Definitions

In [1870]:
ROM_clk = 0   #1= rom com clock || 0= sem clock
ROM_name = f"ROM_fixedpoint_{BIT_WIDTH}bit"


In [1872]:
memory_width= n_integer + n_mantissa_sigmoid
memory_depth = len(df_bin)

if memory_address_type == False : # 0= int(n)
  memory_value_type = 'DEC' #Enter BIN, DEC, HEX, OCT, or UNS; #because memory is with (signed/unsigned) bin address
else: #1= float(address)
  memory_value_type = 'BIN' #Enter BIN, DEC, HEX, OCT, or UNS; #because memory is with (signed/unsigned) bin address

memory_depth

256

In [1873]:
if ROM_clk:
	rom_text_asic = ( 
f'''--https://stackoverflow.com/questions/17579716/implementing-rom-in-xilinx-vhdl
LIBRARY ieee ;
USE ieee.std_logic_1164.all ;
use ieee.numeric_std.all;

----------------

entity {ROM_name} is
generic(addr_width : integer := {str(memory_depth)}; -- store {str(memory_depth)} elements
        addr_bits  : integer := {str(int(math.log2(memory_depth)))}; -- required bits to store {str(memory_depth)} elements
        data_width : integer := {str(memory_width)}  -- each element has {str(memory_width)}-bits
				);
			
port ( clk : in std_logic ;
		address : in std_logic_vector(addr_bits-1 downto 0);
		data_out : out std_logic_vector(data_width-1 downto 0)
		) ;
end entity ;

------------------
architecture arch of {ROM_name} is

	signal reg_address: std_logic_vector(addr_bits-1 downto 0);
	type memory is array ( 0 to addr_width-1 ) of std_logic_vector(data_width-1 downto 0 ) ;
	constant myrom : memory := (
{text_rom_to_asic}
--	2 => "11111111" , --255
--	3 => "11010101" ,  
others => "00000000000" 
) ;

begin 
	process(clk)
		begin 
		if( clk'event and clk = '1' ) then
			reg_address <= address ;
		end if ;
	end process ;
---------------
data_out <= myrom(to_integer(unsigned(reg_address))) ;
end architecture ;

''')

else:
	rom_text_asic = ( f'''

--https://stackoverflow.com/questions/17579716/implementing-rom-in-xilinx-vhdl
LIBRARY ieee ;
USE ieee.std_logic_1164.all ;
use ieee.numeric_std.all;

----------------

ENTITY {ROM_name} is
generic(addr_width : integer := {str(memory_depth)}; -- store {str(memory_depth)} elements
				addr_bits  : integer := {str(int(math.log2(memory_depth)))}; -- required bits to store {str(memory_depth)} elements
				data_width : integer := {str(memory_width)}  -- each element has {str(memory_width)}-bits
				);
			
  PORT (
    address : IN STD_LOGIC_VECTOR(addr_bits - 1 DOWNTO 0);
    data_out : OUT STD_LOGIC_VECTOR(data_width - 1 DOWNTO 0)
  );
END ENTITY;

------------------
architecture arch of {ROM_name} is

	type memory is array ( 0 to addr_width-1 ) of std_logic_vector(data_width-1 downto 0 ) ;
	constant myrom : memory := (
{text_rom_to_asic}
--	2 => "11111111" , --255
--	3 => "11010101" ,  
others => "00000000000" 
) ;

begin 
---------------
data_out <= myrom(to_integer(unsigned(address))) ;
end architecture ;

''')


# print(rom_text_asic)



--https://stackoverflow.com/questions/17579716/implementing-rom-in-xilinx-vhdl
LIBRARY ieee ;
USE ieee.std_logic_1164.all ;
use ieee.numeric_std.all;

----------------

ENTITY ROM_fixedpoint_8bit is
generic(addr_width : integer := 256; -- store 256 elements
				addr_bits  : integer := 8; -- required bits to store 256 elements
				data_width : integer := 8  -- each element has 8-bits
				);
			
  PORT (
    address : IN STD_LOGIC_VECTOR(addr_bits - 1 DOWNTO 0);
    data_out : OUT STD_LOGIC_VECTOR(data_width - 1 DOWNTO 0)
  );
END ENTITY;

------------------
architecture arch of ROM_fixedpoint_8bit is

	type memory is array ( 0 to addr_width-1 ) of std_logic_vector(data_width-1 downto 0 ) ;
	constant myrom : memory := (
--"int_f_x",--(address)  =integer_MAC|| f(x)               = int_f_x 
"10000000",-- (00000000) = 0.0       || 128.0              = 128.0
"10000010",-- (00000001) = 1.0       || 130.79955347088494 = 130.0
"10000101",-- (00000010) = 2.0       || 133.59642981673    = 133.0

In [1874]:
if DOWNLOAD:
    text_file = open(f"{path}{ROM_name}.vhd", "w")
    text_file.write(rom_text_asic)
    text_file.close()